In [1]:
%pip install numpy scipy matplotlib ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [1]:

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import iv
from ipywidgets import interact, FloatSlider

def f(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return -4 * k1 * k2 * np.sin(2 * mu) / kappa
    elif kappa > 1e3:
        return 2 * k1 * (np.cos(2 * mu) - k2) * np.sin(2 * mu)
    else:
        term1 = 1 - 2 * iv(2, kappa / 2) / ((kappa / 2) * iv(1, kappa / 2))
        term2 = 2 / kappa + iv(2, kappa / 2) / iv(1, kappa / 2)
        return 2 * k1 * (term1 * np.cos(2 * mu) - k2 * term2) * np.sin(2 * mu)

# Function g(mu, kappa) for the system
# This function is used to compute the right-hand side of the system of equations
# It is defined based on the parameters mu, kappa, k1, and k2
# The function handles different ranges of kappa to ensure numerical stability
def g(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return 8 * k1 * k2 * np.cos(2 * mu)
    elif kappa > 1e3:
        return 64 * k1 * (k2 * np.cos(2 * mu) - np.cos(4 * mu)) * kappa
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        I2 = iv(2, kappa / 2)
        numerator = 8 * k1 * (k2 * np.cos(2 * mu) - I2 / I1 * np.cos(4 * mu))
        denominator = (kappa / 2) * (I0 / I1 - I1 / I0) - 1
        return numerator / denominator

# Function h(kappa) for the system
def h(kappa):
    if kappa < 1e-2:
        return 2 * kappa
    elif kappa > 1e3:
        return 8 * kappa**2
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        return 4 / (I0 / I1 - I1 / I0 - 2 / kappa)

# RHS of system
def system(mu, kappa, ε, Θ, k1, k2):
    dmu = ε**2 * f(mu, kappa, k1, k2)
    dkappa = ε**2 * g(mu, kappa, k1, k2) - Θ**2 * h(kappa)
    return dmu, dkappa

# Plot phase portrait
def plot_phase_diagram(ε=1.0, Θ=0.5, k1=1.0, k2=-1.0):
    mu_range = np.linspace(0, np.pi, 20)
    kappa_range = np.linspace(0.1, 10.0, 20)
    MU, KAPPA = np.meshgrid(mu_range, kappa_range)

    dMU = np.zeros_like(MU)
    dKAPPA = np.zeros_like(KAPPA)
    MAG = np.zeros_like(MU)

    for i in range(MU.shape[0]):
        for j in range(MU.shape[1]):
            dmu, dkappa = system(MU[i, j], KAPPA[i, j], ε, Θ, k1, k2)
            norm = np.sqrt(dmu**2 + dkappa**2)
            MAG[i, j] = norm + 0.1
            if norm != 0:
                dMU[i, j] = dmu / norm
                dKAPPA[i, j] = dkappa / norm

    plt.figure(figsize=(8, 6))
    plt.quiver(MU, KAPPA, dMU, dKAPPA, scale=30, width=0.003)
    plt.contourf(MU, KAPPA, MAG, levels=50, cmap='YlOrRd', alpha=0.4)
    plt.colorbar(label="Vector Field Magnitude")
    plt.xlabel(r"$\mu$")
    plt.ylabel(r"$\kappa$")
    plt.title("Phase Portrait in the $(\mu, \kappa)$ Plane")
    plt.grid(True)
    plt.xlim(0, np.pi)
    plt.ylim(kappa_range.min(), kappa_range.max())
    plt.show()

# Interactive sliders
interact(
    plot_phase_diagram,
    ε=FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="ε"),
    Θ=FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description=rf"T"),
    k1=FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.1, description="K₁"),
    k2=FloatSlider(value=-1.0, min=-2.0, max=2.0, step=0.1, description="K₂")
)

interactive(children=(FloatSlider(value=1.0, description='ε', max=2.0, min=0.1), FloatSlider(value=0.5, descri…

<function __main__.plot_phase_diagram(ε=1.0, Θ=0.5, k1=1.0, k2=-1.0)>

In [2]:

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import iv
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider

def f(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return -4 * k1 * k2 * np.sin(2 * mu) / kappa
    elif kappa > 1e3:
        return 2 * k1 * (np.cos(2 * mu) - k2) * np.sin(2 * mu)
    else:
        term1 = 1 - 2 * iv(2, kappa / 2) / ((kappa / 2) * iv(1, kappa / 2))
        term2 = 2 / kappa + iv(2, kappa / 2) / iv(1, kappa / 2)
        return 2 * k1 * (term1 * np.cos(2 * mu) - k2 * term2) * np.sin(2 * mu)

# Function g(mu, kappa) for the system
# This function is used to compute the right-hand side of the system of equations
# It is defined based on the parameters mu, kappa, k1, and k2
# The function handles different ranges of kappa to ensure numerical stability
def g(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return 8 * k1 * k2 * np.cos(2 * mu)
    elif kappa > 1e3:
        return 64 * k1 * (k2 * np.cos(2 * mu) - np.cos(4 * mu)) * kappa
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        I2 = iv(2, kappa / 2)
        numerator = 8 * k1 * (k2 * np.cos(2 * mu) - I2 / I1 * np.cos(4 * mu))
        denominator = (kappa / 2) * (I0 / I1 - I1 / I0) - 1
        return numerator / denominator

# Function h(kappa) for the system
def h(kappa):
    if kappa < 1e-2:
        return 2 * kappa
    elif kappa > 1e3:
        return 8 * kappa**2
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        return 4 / (I0 / I1 - I1 / I0 - 2 / kappa)

# RHS of system
def system(mu, kappa, ε, Θ, k1, k2):
    dmu = ε**2 * f(mu, kappa, k1, k2)
    dkappa = ε**2 * g(mu, kappa, k1, k2) - Θ**2 * h(kappa)
    return dmu, dkappa

def system_rhs(t, y, ε, Θ, k1, k2):
    mu, kappa = y
    dmu = ε**2 * f(mu, kappa, k1, k2)
    dkappa = ε**2 * g(mu, kappa, k1, k2) - Θ**2 * h(kappa)
    return [dmu, dkappa]

# Plot phase portrait
def plot_phase_diagram(ε=1.0, Θ=1.0, k1=1.0, k2=-1.0):
    
    t_span = (0, 20)
    t_eval = np.linspace(*t_span, 1000)
    mu0_list = [0.5 * np.pi, 1e-4 , np.pi-1e-4 , 0.02 * np.pi, 0.05 * np.pi, 0.09 * np.pi, 0.14 * np.pi, 0.18 * np.pi, 0.24 * np.pi, 0.3 * np.pi, 0.38 * np.pi, 0.5 * np.pi]
    kappa0_list = [5.0] * len(mu0_list) 
    kappa0_list[0] = 0.01
    # Trajectories
    trajectories = []
    for mu0, kappa0 in zip(mu0_list, kappa0_list):
        sol = solve_ivp(system_rhs, t_span, [mu0, kappa0], args=(ε, Θ, k1, k2), t_eval=t_eval, rtol=1e-6)
        trajectories.append(sol)

    mu_range = np.linspace(0, np.pi, 20)
    kappa_range = np.linspace(0.1, 5.0, 20)
    MU, KAPPA = np.meshgrid(mu_range, kappa_range)

    dMU = np.zeros_like(MU)
    dKAPPA = np.zeros_like(KAPPA)
    MAG = np.zeros_like(MU)

    for i in range(MU.shape[0]):
        for j in range(MU.shape[1]):
            dmu, dkappa = system(MU[i, j], KAPPA[i, j], ε, Θ, k1, k2)
            norm = np.sqrt(dmu**2 + dkappa**2)
            MAG[i, j] = norm + 0.1
            if norm != 0:
                dMU[i, j] = dmu / norm
                dKAPPA[i, j] = dkappa / norm

    

    plt.figure(figsize=(8, 6))
    for sol in trajectories:
        plt.plot(sol.y[0] / np.pi, sol.y[1], lw=1.5, color='blue')
        plt.plot(1 - sol.y[0] / np.pi, sol.y[1], lw=1.5, color='blue')

        MU_traj = sol.y[0][::4]     # shape: (n,)
        KAPPA_traj = sol.y[1][::4]   # shape: (n,)

        dMU_traj = np.zeros_like(MU_traj)
        dKAPPA_traj = np.zeros_like(KAPPA_traj)
        MAG_traj = np.zeros_like(MU_traj)

        for i in range(MU.shape[0]):
            dmu, dkappa = system(MU_traj[i], KAPPA_traj[i], ε, Θ, k1, k2)
            norm = np.sqrt(dmu**2 + dkappa**2)
            MAG_traj[i] = norm + 0.1  # Optional offset for visualization
            if norm != 0:
                dMU_traj[i] = dmu / norm
                dKAPPA_traj[i] = dkappa / norm
        plt.quiver(MU_traj / np.pi, KAPPA_traj, dMU_traj, dKAPPA_traj, scale=45, width=0.015)
        MU_traj = np.pi - sol.y[0][::4]     # shape: (n,)
        KAPPA_traj = sol.y[1][::4]   # shape: (n,)

        dMU_traj = np.zeros_like(MU_traj)
        dKAPPA_traj = np.zeros_like(KAPPA_traj)
        MAG_traj = np.zeros_like(MU_traj)

        for i in range(MU.shape[0]):
            dmu, dkappa = system(MU_traj[i], KAPPA_traj[i], ε, Θ, k1, k2)
            norm = np.sqrt(dmu**2 + dkappa**2)
            MAG_traj[i] = norm + 0.1  # Optional offset for visualization
            if norm != 0:
                dMU_traj[i] = dmu / norm
                dKAPPA_traj[i] = dkappa / norm
        plt.quiver(MU_traj / np.pi, KAPPA_traj, dMU_traj, dKAPPA_traj, scale=45, width=0.015)
    plt.contourf(MU / np.pi, KAPPA, MAG, levels=50, cmap='YlOrRd', alpha=0.4)
    plt.colorbar()
    plt.xlabel(r"$\mu/ \pi$", fontsize = 18)
    plt.ylabel(r"$\kappa$", fontsize = 18)
    #plt.title(r"Solution trajectories in the $(\mu, \kappa)$ plane for $\epsilon = {:.1f}$, $\tau = {:.1f}$, $k_1 = {:.1f}$, $k_2 = {:.1f}$".format(ε, Θ, k1, k2))
    plt.grid(True)
    plt.xlim(-1e-3, 1+1e-3)
    plt.ylim(1e-1, 5)
    plt.show()

# Interactive sliders
interact(
    plot_phase_diagram,
    ε=FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="ε"),
    Θ=FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description=rf"T"),
    k1=FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.1, description="K₁"),
    k2=FloatSlider(value=-1.0, min=-2.0, max=2.0, step=0.1, description="K₂")
)

interactive(children=(FloatSlider(value=1.0, description='ε', max=2.0, min=0.1), FloatSlider(value=1.0, descri…

<function __main__.plot_phase_diagram(ε=1.0, Θ=1.0, k1=1.0, k2=-1.0)>

In [ ]:

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import iv
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider

def f(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return -4 * k1 * k2 * np.sin(2 * mu) / kappa
    elif kappa > 1e3:
        return 2 * k1 * (np.cos(2 * mu) - k2) * np.sin(2 * mu)
    else:
        term1 = 1 - 2 * iv(2, kappa / 2) / ((kappa / 2) * iv(1, kappa / 2))
        term2 = 2 / kappa + iv(2, kappa / 2) / iv(1, kappa / 2)
        return 2 * k1 * (term1 * np.cos(2 * mu) - k2 * term2) * np.sin(2 * mu)

# Function g(mu, kappa) for the system
# This function is used to compute the right-hand side of the system of equations
# It is defined based on the parameters mu, kappa, k1, and k2
# The function handles different ranges of kappa to ensure numerical stability
def g(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return 8 * k1 * k2 * np.cos(2 * mu)
    elif kappa > 1e3:
        return 64 * k1 * (k2 * np.cos(2 * mu) - np.cos(4 * mu)) * kappa
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        I2 = iv(2, kappa / 2)
        numerator = 8 * k1 * (k2 * np.cos(2 * mu) - I2 / I1 * np.cos(4 * mu))
        denominator = (kappa / 2) * (I0 / I1 - I1 / I0) - 1
        return numerator / denominator

# Function h(kappa) for the system
def h(kappa):
    if kappa < 1e-2:
        return 2 * kappa
    elif kappa > 1e3:
        return 8 * kappa**2
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        return 4 / (I0 / I1 - I1 / I0 - 2 / kappa)

# RHS of system
def system(mu, kappa, ε, Θ, k1, k2):
    dmu = ε**2 * f(mu, kappa, k1, k2)
    dkappa = ε**2 * g(mu, kappa, k1, k2) - Θ**2 * h(kappa)
    return dmu, dkappa

def system_rhs(t, y, ε, Θ, k1, k2):
    mu, kappa = y
    dmu = ε**2 * f(mu, kappa, k1, k2)
    dkappa = ε**2 * g(mu, kappa, k1, k2) - Θ**2 * h(kappa)
    return [dmu, dkappa]

# Plot phase portrait
def plot_phase_diagram(ε=1.0, Θ=1.0, k1=1.0, k2=-1.0):
    

    mu_range = np.linspace(0, np.pi, 40)
    kappa_range = np.linspace(0.1, 10.0, 40)
    MU, KAPPA = np.meshgrid(mu_range, kappa_range)

    dMU = np.zeros_like(MU)
    dKAPPA = np.zeros_like(KAPPA)
    dmu = np.zeros_like(MU)
    dkappa = np.zeros_like(MU)
    MAG = np.zeros_like(MU)

    for i in range(MU.shape[0]):
        for j in range(MU.shape[1]):
            dmu[i, j], dkappa[i, j] = system(MU[i, j], KAPPA[i, j], ε, Θ, k1, k2)
            norm = np.sqrt(dmu[i, j]**2 + dkappa[i, j]**2)
            MAG[i, j] = norm + 0.1
            if norm != 0:
                dMU[i, j] = dmu[i, j] / norm
                dKAPPA[i, j] = dkappa[i, j] / norm
    
    plt.figure(figsize=(8, 6))
    
    plt.contour(MU / np.pi, KAPPA, dmu, levels=[0], colors='blue', linewidths=2, linestyles='-')
    plt.contour(MU / np.pi, KAPPA, dkappa, levels=[0], colors='red', linewidths=2, linestyles='--')
    plt.axvline(1, color='blue', linestyle='-', linewidth=3, label=r'$\mu = \pi$')  # Add this line
    plt.xlabel(r"$\mu/\pi$",fontsize = 18)
    plt.ylabel(r"$\kappa$", fontsize = 18)


    plt.grid(True)
    plt.xlim(-1e-3, 1+1e-3)
    plt.ylim(1e-1, 5)
    # Add labels for the four regions
    plt.text(0.5 / np.pi, 4.0, r"$\dot\mu > 0, \dot\kappa < 0$", fontsize=14, color="black")
    plt.text(2.2 / np.pi, 4.0, r"$\dot\mu < 0, \dot\kappa < 0$", fontsize=14, color="black")
    plt.text(0.9 / np.pi, 0.8, r"$\dot\mu > 0, \dot\kappa > 0$", fontsize=14, color="black")
    plt.text(1.7 / np.pi, 0.8, r"$\dot\mu < 0, \dot\kappa > 0$", fontsize=14, color="black")
    plt.savefig("phaseportrait2.png", dpi=300)
    plt.show()

# Interactive sliders
interact(
    plot_phase_diagram,
    ε=FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="ε"),
    Θ=FloatSlider(value=0.9, min=0.0, max=2.0, step=0.1, description=rf"T"),
    k1=FloatSlider(value=0.4, min=-2.0, max=2.0, step=0.1, description="K₁"),
    k2=FloatSlider(value=-2.0, min=-2.0, max=2.0, step=0.1, description="K₂")
)

interactive(children=(FloatSlider(value=1.0, description='ε', max=2.0, min=0.1), FloatSlider(value=0.9, descri…

<function __main__.plot_phase_diagram(ε=1.0, Θ=1.0, k1=1.0, k2=-1.0)>

In [5]:

# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import iv
from scipy.integrate import solve_ivp
from ipywidgets import interact, FloatSlider

def f(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return -4 * k1 * k2 * np.sin(2 * mu) / kappa
    elif kappa > 1e3:
        return 2 * k1 * (np.cos(2 * mu) - k2) * np.sin(2 * mu)
    else:
        term1 = 1 - 2 * iv(2, kappa / 2) / ((kappa / 2) * iv(1, kappa / 2))
        term2 = 2 / kappa + iv(2, kappa / 2) / iv(1, kappa / 2)
        return 2 * k1 * (term1 * np.cos(2 * mu) - k2 * term2) * np.sin(2 * mu)

# Function g(mu, kappa) for the system
# This function is used to compute the right-hand side of the system of equations
# It is defined based on the parameters mu, kappa, k1, and k2
# The function handles different ranges of kappa to ensure numerical stability
def g(mu, kappa, k1, k2):
    if kappa < 1e-2:
        return 8 * k1 * k2 * np.cos(2 * mu)
    elif kappa > 1e3:
        return 64 * k1 * (k2 * np.cos(2 * mu) - np.cos(4 * mu)) * kappa
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        I2 = iv(2, kappa / 2)
        numerator = 8 * k1 * (k2 * np.cos(2 * mu) - I2 / I1 * np.cos(4 * mu))
        denominator = (kappa / 2) * (I0 / I1 - I1 / I0) - 1
        return numerator / denominator

# Function h(kappa) for the system
def h(kappa):
    if kappa < 1e-2:
        return 2 * kappa
    elif kappa > 1e3:
        return 8 * kappa**2
    else:
        I0 = iv(0, kappa / 2)
        I1 = iv(1, kappa / 2)
        return 4 / (I0 / I1 - I1 / I0 - 2 / kappa)

# RHS of system
def system(mu, kappa, ε, Θ, k1, k2):
    dmu = ε**2 * f(mu, kappa, k1, k2)
    dkappa = ε**2 * g(mu, kappa, k1, k2) - Θ**2 * h(kappa)
    return dmu, dkappa

def system_rhs(t, y, ε, Θ, k1, k2):
    mu, kappa = y
    dmu = ε**2 * f(mu, kappa, k1, k2)
    dkappa = ε**2 * g(mu, kappa, k1, k2) - Θ**2 * h(kappa)
    return [dmu, dkappa]

# Plot phase portrait
def plot_phase_diagram(ε=1.0, Θ=1.0, k1=1.0, k2=-1.0, mu0overpi=0.5, kappa0=5.0):
    
    mu0 = mu0overpi * np.pi
    t_span = (0, 20)
    t_eval = np.linspace(*t_span, 1000)
    mu0_list = [mu0]
    kappa0_list = [kappa0]
    # Trajectories
    trajectories = []
    for mu0, kappa0 in zip(mu0_list, kappa0_list):
        sol = solve_ivp(system_rhs, t_span, [mu0, kappa0], args=(ε, Θ, k1, k2), t_eval=t_eval, rtol=1e-6)
        trajectories.append(sol)

    mu_range = np.linspace(0, np.pi, 20)
    kappa_range = np.linspace(0.1, 5.0, 20)
    MU, KAPPA = np.meshgrid(mu_range, kappa_range)

    dMU = np.zeros_like(MU)
    dKAPPA = np.zeros_like(KAPPA)
    MAG = np.zeros_like(MU)

    for i in range(MU.shape[0]):
        for j in range(MU.shape[1]):
            dmu, dkappa = system(MU[i, j], KAPPA[i, j], ε, Θ, k1, k2)
            norm = np.sqrt(dmu**2 + dkappa**2)
            MAG[i, j] = norm + 0.1
            if norm != 0:
                dMU[i, j] = dmu / norm
                dKAPPA[i, j] = dkappa / norm

    

    plt.figure(figsize=(8, 6))
    for sol in trajectories:
        plt.plot(sol.y[0] / np.pi, sol.y[1], lw=1.5, color='blue')
        #plt.plot(1 - sol.y[0] / np.pi, sol.y[1], lw=1.5, color='blue')

        MU_traj = sol.y[0][::4]     # shape: (n,)
        KAPPA_traj = sol.y[1][::4]   # shape: (n,)

        dMU_traj = np.zeros_like(MU_traj)
        dKAPPA_traj = np.zeros_like(KAPPA_traj)
        MAG_traj = np.zeros_like(MU_traj)

        for i in range(MU.shape[0]):
            dmu, dkappa = system(MU_traj[i], KAPPA_traj[i], ε, Θ, k1, k2)
            norm = np.sqrt(dmu**2 + dkappa**2)
            MAG_traj[i] = norm + 0.1  # Optional offset for visualization
            if norm != 0:
                dMU_traj[i] = dmu / norm
                dKAPPA_traj[i] = dkappa / norm
        plt.quiver(MU_traj / np.pi, KAPPA_traj, dMU_traj, dKAPPA_traj, scale=45, width=0.015)
        MU_traj = np.pi - sol.y[0][::4]     # shape: (n,)
        KAPPA_traj = sol.y[1][::4]   # shape: (n,)

    
    plt.xlabel(r"$\mu/ \pi$", fontsize = 18)
    plt.ylabel(r"$\kappa$", fontsize = 18)
    #plt.title(r"Solution trajectories in the $(\mu, \kappa)$ plane for $\epsilon = {:.1f}$, $\tau = {:.1f}$, $k_1 = {:.1f}$, $k_2 = {:.1f}$".format(ε, Θ, k1, k2))
    plt.grid(True)
    plt.xlim(-1e-3, 1+1e-3)
    plt.ylim(1e-1, 5)
    plt.savefig("phaseportrait-two-stage.png", dpi=300)
    plt.show()

# Interactive sliders
interact(
    plot_phase_diagram,
    ε=FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description="ε"),
    Θ=FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description=rf"T"),
    k1=FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.1, description="K₁"),
    k2=FloatSlider(value=-1.0, min=-2.0, max=2.0, step=0.1, description="K₂"),
    mu0overpi=FloatSlider(value=0.5, min=0, max=1, step=0.01, description="$\\mu_0/\\pi$"),
    kappa0=FloatSlider(value=5.0, min=0.1, max=5.0, step=0.1, description="$\\kappa_0$")
)

interactive(children=(FloatSlider(value=1.0, description='ε', max=2.0, min=0.1), FloatSlider(value=1.0, descri…

<function __main__.plot_phase_diagram(ε=1.0, Θ=1.0, k1=1.0, k2=-1.0, mu0overpi=0.5, kappa0=5.0)>